In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

In [83]:
df = pd.read_csv('Unit_1.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df['Hydraulic_head'] = df.Magat_level - df.Maris_level

#### Univariate Regression using DNN

In [104]:
X = np.array(df['Hydraulic_head'])
y = df['U1_ActivePower']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

In [106]:
normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=None)
normalizer.adapt(X_train)

In [107]:
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [108]:
model.compile(loss='mse', optimizer=Adam(0.001), metrics=['mae'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 1)                3         
 ion)                                                            
                                                                 
 dense_68 (Dense)            (None, 64)                128       
                                                                 
 dense_69 (Dense)            (None, 64)                4160      
                                                                 
 dense_70 (Dense)            (None, 1)                 65        
                                                                 
Total params: 4,356
Trainable params: 4,353
Non-trainable params: 3
_________________________________________________________________


In [109]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=20 )

Epoch 1/20
636/636 [==============================] - 2s 2ms/step - loss: 1355.7616 - mae: 23.1097 - val_loss: 15.2504 - val_mae: 2.5056
Epoch 2/20
636/636 [==============================] - 1s 2ms/step - loss: 6.2180 - mae: 1.4491 - val_loss: 2.0058 - val_mae: 0.8973
Epoch 3/20
636/636 [==============================] - 1s 2ms/step - loss: 1.5098 - mae: 0.7639 - val_loss: 1.0553 - val_mae: 0.6667
Epoch 4/20
636/636 [==============================] - 1s 2ms/step - loss: 1.0393 - mae: 0.6380 - val_loss: 0.9046 - val_mae: 0.5929
Epoch 5/20
636/636 [==============================] - 1s 2ms/step - loss: 0.9618 - mae: 0.6064 - val_loss: 0.8763 - val_mae: 0.5918
Epoch 6/20
636/636 [==============================] - 1s 2ms/step - loss: 0.9479 - mae: 0.5979 - val_loss: 0.8666 - val_mae: 0.5874
Epoch 7/20
636/636 [==============================] - 1s 2ms/step - loss: 0.9454 - mae: 0.5964 - val_loss: 0.8659 - val_mae: 0.5886
Epoch 8/20
636/636 [==============================] - 1s 2ms/step - los

#### Multivariate Regression using DNN

In [110]:
X = df.drop('U1_ActivePower', axis=1)
y = df['U1_ActivePower']

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

In [112]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [113]:
model = Sequential()
model.add(Dense(128, input_dim=5, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [114]:
model.compile(loss='mse', optimizer=Adam(0.0001), metrics=['mae'])
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 128)               768       
                                                                 
 dense_72 (Dense)            (None, 64)                8256      
                                                                 
 dense_73 (Dense)            (None, 64)                4160      
                                                                 
 dense_74 (Dense)            (None, 1)                 65        
                                                                 
Total params: 13,249
Trainable params: 13,249
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=300, batch_size=128)

Epoch 1/300


c:\Users\deguz\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


159/159 [==============================] - 2s 4ms/step - loss: 6918.5093 - mae: 83.1531 - val_loss: 6616.0747 - val_mae: 81.3136
Epoch 2/300
159/159 [==============================] - 0s 3ms/step - loss: 5638.5352 - mae: 74.8158 - val_loss: 4196.2388 - val_mae: 64.3678
Epoch 3/300
159/159 [==============================] - 0s 3ms/step - loss: 2386.6355 - mae: 45.8980 - val_loss: 1002.9837 - val_mae: 28.2223
Epoch 4/300
159/159 [==============================] - 0s 2ms/step - loss: 679.6467 - mae: 21.7927 - val_loss: 541.9293 - val_mae: 19.1059
Epoch 5/300
159/159 [==============================] - 0s 2ms/step - loss: 510.3015 - mae: 18.6817 - val_loss: 463.8701 - val_mae: 17.8344
Epoch 6/300
159/159 [==============================] - 0s 3ms/step - loss: 445.9137 - mae: 17.4476 - val_loss: 405.7924 - val_mae: 16.6240
Epoch 7/300
159/159 [==============================] - 0s 2ms/step - loss: 391.9080 - mae: 16.2723 - val_loss: 354.1948 - val_mae: 15.4132
Epoch 8/300
159/159 [============

#### Hyperparameter tuning using Keras Tuner

In [9]:
def model_builder(hp):
  model = tf.keras.Sequential()
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units2', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units1, input_dim=5, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(tf.keras.layers.Dense(1))
  
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['mae'])

  return model

In [10]:
tuner = kt.Hyperband(model_builder,
                     objective='val_mae',
                     max_epochs=50,
                     directory='my_dir',
                     project_name='magat_kt')

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=7)

In [12]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 90 Complete [00h 00m 42s]
val_mae: 0.38230156898498535

Best val_mae So Far: 0.3229147791862488
Total elapsed time: 00h 16m 38s
INFO:tensorflow:Oracle triggered exit


In [13]:
for h_param in [f"units{i}" for i in range(1,3)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units1 480
units2 416
learning_rate 0.0001


#### Retraining

In [66]:
model = Sequential()
model.add(Dense(480, input_dim=5, activation='relu'))
model.add(Dense(416, activation='relu'))
model.add(Dense(1))

In [67]:
model.compile(loss='mse', optimizer=Adam(0.0001), metrics=['mae'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 480)               2880      
                                                                 
 dense_47 (Dense)            (None, 416)               200096    
                                                                 
 dense_48 (Dense)            (None, 1)                 417       
                                                                 
Total params: 203,393
Trainable params: 203,393
Non-trainable params: 0
_________________________________________________________________


In [68]:
checkpointer = ModelCheckpoint(filepath='checkpoint_dir/training_0.ckpt', 
                               monitor='val_mae', mode='min',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_mae', mode='min', verbose=1, patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.1, patience=5, min_lr=0.00001, verbose=1)
callbacks=[early_stopping, checkpointer]

In [69]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=callbacks)

Epoch 1/1000


c:\Users\deguz\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


628/636 [============================>.] - ETA: 0s - loss: 2780.9497 - mae: 45.0440
Epoch 1: val_mae improved from inf to 17.32417, saving model to checkpoint_dir\training_0.ckpt
INFO:tensorflow:Assets written to: checkpoint_dir\training_0.ckpt\assets
636/636 [==============================] - 2s 3ms/step - loss: 2752.7922 - mae: 44.7218 - val_loss: 440.4384 - val_mae: 17.3242
Epoch 2/1000
620/636 [============================>.] - ETA: 0s - loss: 315.9128 - mae: 14.3881
Epoch 2: val_mae improved from 17.32417 to 11.03116, saving model to checkpoint_dir\training_0.ckpt
INFO:tensorflow:Assets written to: checkpoint_dir\training_0.ckpt\assets
636/636 [==============================] - 2s 3ms/step - loss: 312.9120 - mae: 14.3086 - val_loss: 190.1340 - val_mae: 11.0312
Epoch 3/1000
631/636 [============================>.] - ETA: 0s - loss: 122.1625 - mae: 8.5909
Epoch 3: val_mae improved from 11.03116 to 6.09463, saving model to checkpoint_dir\training_0.ckpt
INFO:tensorflow:Assets written

#### Going Deeper

In [115]:
model = Sequential()
model.add(Dense(128, input_dim=5, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1))

In [72]:
model.compile(loss='mse', optimizer=Adam(0.01), metrics=['mae'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 128)               768       
                                                                 
 dense_55 (Dense)            (None, 256)               33024     
                                                                 
 dense_56 (Dense)            (None, 512)               131584    
                                                                 
 dense_57 (Dense)            (None, 1024)              525312    
                                                                 
 dense_58 (Dense)            (None, 1)                 1025      
                                                                 
Total params: 691,713
Trainable params: 691,713
Non-trainable params: 0
_________________________________________________________________


In [74]:
checkpointer = ModelCheckpoint(filepath='checkpoint_dir/training_1.ckpt', 
                               monitor='val_mae', mode='min',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.1, patience=3, min_lr=0.00001, verbose=1)
callbacks=[early_stopping, checkpointer, reduce_lr]

In [75]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=callbacks)

Epoch 1/1000


c:\Users\deguz\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


631/636 [============================>.] - ETA: 0s - loss: 50.8453 - mae: 2.8147
Epoch 1: val_mae improved from inf to 0.76234, saving model to checkpoint_dir\training_1.ckpt
INFO:tensorflow:Assets written to: checkpoint_dir\training_1.ckpt\assets
636/636 [==============================] - 2s 3ms/step - loss: 50.4792 - mae: 2.8042 - val_loss: 1.2837 - val_mae: 0.7623 - lr: 0.0100
Epoch 2/1000
622/636 [============================>.] - ETA: 0s - loss: 3.3530 - mae: 1.2332
Epoch 2: val_mae did not improve from 0.76234
636/636 [==============================] - 2s 2ms/step - loss: 3.2974 - mae: 1.2195 - val_loss: 1.7336 - val_mae: 1.2003 - lr: 0.0100
Epoch 3/1000
623/636 [============================>.] - ETA: 0s - loss: 1.8002 - mae: 0.9795
Epoch 3: val_mae did not improve from 0.76234
636/636 [==============================] - 1s 2ms/step - loss: 1.8028 - mae: 0.9808 - val_loss: 1.2577 - val_mae: 0.9462 - lr: 0.0100
Epoch 4/1000
635/636 [============================>.] - ETA: 0s - loss:

#### Loading the saved weights

In [118]:
model = Sequential()
model.add(Dense(128, input_dim=5, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1))

In [119]:
model.compile(loss='mse', optimizer=Adam(0.01), metrics=['mae'])

In [120]:
model.load_weights('checkpoint_dir/training_1.ckpt')

In [121]:
score = model.evaluate(X_test, y_test)

341/341 [==============================] - 1s 2ms/step - loss: 0.5603 - mae: 0.3266


#### Saving the Model

In [122]:
model.save('saved_model/dnn_unit1')

INFO:tensorflow:Assets written to: saved_model/dnn_unit1\assets


In [123]:
new_model = tf.keras.models.load_model('saved_model/dnn_unit1')


In [124]:
new_model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 128)               768       
                                                                 
 dense_81 (Dense)            (None, 256)               33024     
                                                                 
 dense_82 (Dense)            (None, 512)               131584    
                                                                 
 dense_83 (Dense)            (None, 1024)              525312    
                                                                 
 dense_84 (Dense)            (None, 1)                 1025      
                                                                 
Total params: 691,713
Trainable params: 691,713
Non-trainable params: 0
_________________________________________________________________


In [125]:
new_model.evaluate(X_test, y_test)

341/341 [==============================] - 1s 2ms/step - loss: 0.5603 - mae: 0.3266


[0.560325562953949, 0.3265897035598755]